In [48]:
from __future__ import annotations

import math

from abc import ABC

from contextlib import nullcontext
from copy import deepcopy

from typing import Dict, Optional, Tuple, Union

import torch
from botorch.acquisition.acquisition import AcquisitionFunction
from botorch.acquisition.objective import PosteriorTransform
from botorch.exceptions import UnsupportedError
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.gpytorch import GPyTorchModel
from botorch.models.model import Model
from botorch.utils.constants import get_constants_like
from botorch.utils.probability import MVNXPB
from botorch.utils.probability.utils import (
    log_ndtr as log_Phi,
    log_phi,
    log_prob_normal_in,
    ndtr as Phi,
    phi,
)
from botorch.utils.safe_math import log1mexp, logmeanexp
from botorch.utils.transforms import convert_to_target_pre_hook, t_batch_mode_transform
from torch import Tensor
from torch.nn.functional import pad

_sqrt_2pi = math.sqrt(2 * math.pi)
# the following two numbers are needed for _log_ei_helper
_neg_inv_sqrt2 = -(2**-0.5)
_log_sqrt_pi_div_2 = math.log(math.pi / 2) / 2


from botorch.acquisition import AnalyticAcquisitionFunction
from botorch.models.model import Model
from typing import Any, Optional, Union
from torch import Tensor
from botorch.acquisition.objective import PosteriorTransform
from botorch.utils.transforms import convert_to_target_pre_hook, t_batch_mode_transform
# from botorch.acquisition import *
from abc import ABC
from typing import Dict, Optional, Tuple, Union
import botorch.acquisition as temp
import botorch.acquisition 
from botorch.utils.probability.utils import log_phi, log_ndtr,ndtr as Phi, phi

# def _gamma(
#     mean: Tensor, sigma: Tensor, fstar: Tensor, maximize: bool
# ) -> Tensor:
#     """Returns `u = (mean - best_f) / sigma`, -u if maximize == True."""
#     u = (fstar - mean) / sigma
#     return u if maximize else -u



class AnalyticAcquisitionFunction(AcquisitionFunction, ABC):
    r"""
    Base class for analytic acquisition functions.

    :meta private:
    """

    def __init__(
        self,
        model: Model,
        posterior_transform: Optional[PosteriorTransform] = None,
        **kwargs,
    ) -> None:
        r"""Base constructor for analytic acquisition functions.

        Args:
            model: A fitted single-outcome model.
            posterior_transform: A PosteriorTransform. If using a multi-output model,
                a PosteriorTransform that transforms the multi-output posterior into a
                single-output posterior is required.
        """
        super().__init__(model=model)
        posterior_transform = self._deprecate_acqf_objective(
            posterior_transform=posterior_transform,
            objective=kwargs.get("objective"),
        )
        if posterior_transform is None:
            if model.num_outputs != 1:
                raise UnsupportedError(
                    "Must specify a posterior transform when using a "
                    "multi-output model."
                )
        else:
            if not isinstance(posterior_transform, PosteriorTransform):
                raise UnsupportedError(
                    "AnalyticAcquisitionFunctions only support PosteriorTransforms."
                )
        self.posterior_transform = posterior_transform

    def set_X_pending(self, X_pending: Optional[Tensor] = None) -> None:
        raise UnsupportedError(
            "Analytic acquisition functions do not account for X_pending yet."
        )

    def _mean_and_sigma(
        self, X: Tensor, compute_sigma: bool = True, min_var: float = 1e-12
    ) -> Tuple[Tensor, Optional[Tensor]]:
        """Computes the first and second moments of the model posterior.

        Args:
            X: `batch_shape x q x d`-dim Tensor of model inputs.
            compute_sigma: Boolean indicating whether or not to compute the second
                moment (default: True).
            min_var: The minimum value the variance is clamped too. Should be positive.

        Returns:
            A tuple of tensors containing the first and second moments of the model
            posterior. Removes the last two dimensions if they have size one. Only
            returns a single tensor of means if compute_sigma is True.
        """
        self.to(device=X.device)  # ensures buffers / parameters are on the same device
        posterior = self.model.posterior(
            X=X, posterior_transform=self.posterior_transform
        )
        mean = posterior.mean.squeeze(-2).squeeze(-1)  # removing redundant dimensions
        if not compute_sigma:
            return mean, None
        sigma = posterior.variance.clamp_min(min_var).sqrt().view(mean.shape)
        return mean, sigma


# --------------- Helper functions for analytic acquisition functions. ---------------


def _scaled_improvement(
    mean: Tensor, sigma: Tensor, best_f: Tensor, maximize: bool
) -> Tensor:
    """Returns `u = (mean - best_f) / sigma`, -u if maximize == True."""
    u = (mean - best_f) / sigma
    return u if maximize else -u


def _gamma(
    mean: Tensor, sigma: Tensor, fstar: Tensor, maximize: bool
) -> Tensor:
    """Returns `u = (mean - best_f) / sigma`, -u if maximize == True."""
    u = (fstar - mean) / sigma
    return u if maximize else -u


def _ei_helper(u: Tensor) -> Tensor:
    """Computes phi(u) + u * Phi(u), where phi and Phi are the standard normal
    pdf and cdf, respectively. This is used to compute Expected Improvement.
    """
    return phi(u) + u * Phi(u)



from torch.nn.functional import softplus
import math

from typing import Callable, Tuple, Union

def cauchy(x: Tensor) -> Tensor:
    """Computes a Lorentzian, i.e. an un-normalized Cauchy density function."""
    return 1 / (1 + x.square())

def fatplus(x: Tensor, tau: Union[float, Tensor] = 0.5) -> Tensor:
    """Computes a fat-tailed approximation to `ReLU(x) = max(x, 0)` by linearly
    combining a regular softplus function and the density function of a Cauchy
    distribution. The coefficient `alpha` of the Cauchy density is chosen to guarantee
    monotonicity and convexity.

    Args:
        x: A Tensor on whose values to compute the smoothed function.
        tau: Temperature parameter controlling the smoothness of the approximation.

    Returns:
        A Tensor of values of the fat-tailed softplus.
    """

    def _fatplus(x: Tensor) -> Tensor:
        alpha = 1e-1  # guarantees monotonicity and convexity (TODO: ref + Lemma 4)
        return softplus(x) + alpha * cauchy(x)

    return tau * _fatplus(x / tau)

    


class ExpectedImprovement(AnalyticAcquisitionFunction):
    r"""Single-outcome Expected Improvement (analytic).

    Computes classic Expected Improvement over the current best observed value,
    using the analytic formula for a Normal posterior distribution. Unlike the
    MC-based acquisition functions, this relies on the posterior at single test
    point being Gaussian (and require the posterior to implement `mean` and
    `variance` properties). Only supports the case of `q=1`. The model must be
    single-outcome.

    `EI(x) = E(max(f(x) - best_f, 0)),`

    where the expectation is taken over the value of stochastic function `f` at `x`.

    Example:
        >>> model = SingleTaskGP(train_X, train_Y)
        >>> EI = ExpectedImprovement(model, best_f=0.2)
        >>> ei = EI(test_X)

    NOTE: It is *strongly* recommended to use LogExpectedImprovement instead of regular
    EI, because it solves the vanishing gradient problem by taking special care of
    numerical computations and can lead to substantially improved BO performance.
    """

    def __init__(
        self,
        model: Model,
        best_f: Union[float, Tensor],
        posterior_transform: Optional[PosteriorTransform] = None,
        maximize: bool = True,
        **kwargs,
    ):
        r"""Single-outcome Expected Improvement (analytic).

        Args:
            model: A fitted single-outcome model.
            best_f: Either a scalar or a `b`-dim Tensor (batch mode) representing
                the best function value observed so far (assumed noiseless).
            posterior_transform: A PosteriorTransform. If using a multi-output model,
                a PosteriorTransform that transforms the multi-output posterior into a
                single-output posterior is required.
            maximize: If True, consider the problem a maximization problem.
        """
        super().__init__(model=model, posterior_transform=posterior_transform, **kwargs)
        self.register_buffer("best_f", torch.as_tensor(best_f))
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:

        r"""Evaluate Expected Improvement on the candidate set X.

        Args:
            X: A `(b1 x ... bk) x 1 x d`-dim batched tensor of `d`-dim design points.
                Expected Improvement is computed for each point individually,
                i.e., what is considered are the marginal posteriors, not the
                joint.

        Returns:
            A `(b1 x ... bk)`-dim tensor of Expected Improvement values at the
            given design points `X`.
        """
        mean, sigma = self._mean_and_sigma(X)
        u = _scaled_improvement(mean, sigma, self.best_f, self.maximize)
        return sigma * _ei_helper(u)

  
  

class Fstar_pdf_GradientEnhanced(AnalyticAcquisitionFunction):
   
    def __init__(
        self,
        model: Model,
        #best_f: Union[float, Tensor],
        fstar: Union[float, Tensor],
        posterior_transform: Optional[PosteriorTransform] = None,
        maximize: bool = True,
        **kwargs,
    ):
        
        
        super().__init__(model=model, posterior_transform=posterior_transform, **kwargs)
        #self.register_buffer("best_f", torch.as_tensor(best_f))
        self.register_buffer("fstar", torch.as_tensor(fstar))
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
       
        mean, sigma = self._mean_and_sigma(X)
        gamma = _gamma(mean, sigma, self.fstar, self.maximize)
        
        
        # use fat softplus to make the value smaller than -15 be -15
        max_dis = 20.
        gamma = fatplus(gamma+max_dis,tau=0.2)-torch.as_tensor(max_dis) 
        gamma = - (fatplus(-gamma+max_dis,tau=0.2)-torch.as_tensor(max_dis))
        
        part1 = log_phi(gamma)
        
        # part 2 calculation
        D = X.shape[-1]
        mean_d, variance_d = self.model.posterior_derivative(X)
        
        logpdf_total = torch.zeros(X.shape[0])
        
        for d in range(D):
            mean_list = mean_d[:,d]
            sigma_list = torch.sqrt(variance_d[:,d,d])
            
            u = (torch.as_tensor(0.)-mean_list)/sigma_list
            
            
            # use fat softplus to make the value smaller than -15 be -15
            max_dis = 20.
            u = fatplus(u+max_dis,tau=0.2)-torch.as_tensor(max_dis) 
            u = - (fatplus(-u+max_dis,tau=0.2)-torch.as_tensor(max_dis))

            
            pdf_temp = log_phi(u)  #log_phi(u)  phi(u)
            
            logpdf_total += pdf_temp
            
            # print(pdf_temp)
            
        #print('log pdf is: ', pdf_total)
        # print('part 2: ',logpdf_total)
        # print(logpdf_total.shape)
        
        part2 = logpdf_total
       
        return part1  + part2
    


class Fstar_pdf_GradientEnhanced_fantasy(AnalyticAcquisitionFunction):
   
    def __init__(
        self,
        model: Model,
        #best_f: Union[float, Tensor],
        fstar: Union[float, Tensor],
        posterior_transform: Optional[PosteriorTransform] = None,
        maximize: bool = True,
        **kwargs,
    ):
        
        
        super().__init__(model=model, posterior_transform=posterior_transform, **kwargs)
        #self.register_buffer("best_f", torch.as_tensor(best_f))
        self.register_buffer("fstar", torch.as_tensor(fstar))
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
       
        mean, sigma = self._mean_and_sigma(X)
        gamma = _gamma(mean, sigma, self.fstar, self.maximize)
        
        
        # use fat softplus to make the value smaller than -15 be -15
        max_dis = 20.
        gamma = fatplus(gamma+max_dis,tau=0.2)-torch.as_tensor(max_dis) 
        gamma = - (fatplus(-gamma+max_dis,tau=0.2)-torch.as_tensor(max_dis))
        
        part1 = log_phi(gamma)
        
        # part 2 calculation
        logpdf_total = torch.zeros(X.shape[0])
        D = X.shape[-1]
        
        
        for ii in range(X.shape[0]):
            
            x_temp = X[ii]
            
            logpdf_temp = 0.
            
            x_temp = x_temp.reshape(1,1,D)
            
            model_temp = self.model.get_fantasy_model(x_temp.reshape(-1,D), torch.tensor([self.fstar.item()]).reshape(-1,1))
            model_temp.N = self.model.N+1
            model_temp.train_targets = model_temp.train_targets.reshape(model_temp.N)
            
            mean_d, variance_d = model_temp.posterior_derivative(x_temp.reshape(-1,1,D))
        
       
            for d in range(D):
                mean_list = mean_d[:,d]
                sigma_list = torch.sqrt(variance_d[:,d,d])
                
                u = (torch.as_tensor(0.)-mean_list)/sigma_list
                
                
                # use fat softplus to make the value smaller than -15 be -15
                max_dis = 20.
                u = fatplus(u+max_dis,tau=0.2)-torch.as_tensor(max_dis) 
                u = - (fatplus(-u+max_dis,tau=0.2)-torch.as_tensor(max_dis))

                
                pdf_temp = log_phi(u)  #log_phi(u)  phi(u)
                
                logpdf_temp += pdf_temp.item()
                
            logpdf_total[ii] = logpdf_temp
            
            # print(pdf_temp)
            
        #print('log pdf is: ', pdf_total)
        # print('part 2: ',logpdf_total)
        # print(logpdf_total.shape)
        
        part2 = logpdf_total
       
        return part1  + part2


In [3]:
import torch
import numpy as np
from botorch.models import SingleTaskGP
from botorch.test_functions import Ackley,Beale,Branin,Rosenbrock,SixHumpCamel,Hartmann,Powell,DixonPrice,Levy,StyblinskiTang,Griewank
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from botorch.utils.transforms import unnormalize,normalize
from torch.quasirandom import SobolEngine
import matplotlib.pyplot as plt
from model import DerivativeExactGPSEModel

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

torch.set_default_dtype(dtype)

In [5]:
def get_initial_points(dim, n_pts, seed=0):
    sobol = SobolEngine(dimension=dim, scramble=True, seed=seed)
    X_init = sobol.draw(n=n_pts).to(dtype=dtype, device=device)
    return X_init

In [51]:
N = 12
iter_num = 30

function = Levy(dim=2,negate=True)
fstar= 0.
bounds=function.bounds.to(device)
dim = bounds.shape[1]

standard_bounds=torch.tensor([0.,1.]*dim).reshape(-1,2).T.to(device)

In [56]:
EI_record = []

for exp in range(1):

  print(exp)
  torch.manual_seed(exp)

  train_x_standard = get_initial_points(dim,4*dim,exp).to(device)
  train_x = unnormalize(train_x_standard, bounds).reshape(-1,dim)
  train_obj = function(train_x).unsqueeze(-1)

  best_value = train_obj.max().item()
  best_value_holder = [best_value]

  for i in range (iter_num):

    train_x_standard = normalize(train_x, bounds).to(device)
    train_obj_standard = (train_obj - train_obj.mean()) / train_obj.std()
    
    fstar_standard = (fstar - train_obj.mean()) / train_obj.std()

    torch.manual_seed(exp+iter_num)
    model = DerivativeExactGPSEModel(dim,train_x_standard, train_obj_standard).to(device)

    #model = SingleTaskGP(train_x_standard, train_obj_standard).to(device)
    

    mll = ExactMarginalLogLikelihood(model.likelihood, model) .to(device)

    torch.manual_seed(exp+iter_num)
    fit_gpytorch_mll(mll)
    
    # print('lengthscale: ',model.covar_module.base_kernel.lengthscale)
    # print('variance: ',model.covar_module.outputscale )
    
    torch.manual_seed(exp+iter_num)
    
    #AF = ExpectedImprovement(model=model, best_f=train_obj_standard.max().item()) .to(device)
    
    #AF = Fstar_pdf_GradientEnhanced(model=model, fstar=fstar_standard) .to(device)

    AF = Fstar_pdf_GradientEnhanced_fantasy(model=model, fstar=fstar_standard) .to(device)

    new_point_analytic, af_val = optimize_acqf(
        acq_function=AF,
        bounds=standard_bounds .to(device),
        q=1,
        num_restarts=3*dim,
        raw_samples=30*dim,
        options={},
    )

    print(new_point_analytic)
    print(af_val)

    next_x = unnormalize(new_point_analytic, bounds).reshape(-1,dim)
    new_obj = function(next_x).unsqueeze(-1) .to(device)


    train_x = torch.cat((train_x, next_x))
    train_obj = torch.cat((train_obj, new_obj))

    best_value = train_obj.max().item()
    best_value_holder.append(best_value)

    print(best_value_holder[-1])

  best_value_holder = np.array(best_value_holder)
  EI_record.append(best_value_holder)

0


/home/hanyang/anaconda3/envs/known_boundary/lib/python3.9/site-packages/botorch/optim/optimize.py:369: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)
/home/hanyang/anaconda3/envs/known_boundary/lib/python3.9/site-packages/botorch/optim/optimize.py:393: RuntimeWarning: Optimization failed on the second try, after generating a new set of initial conditions.
  warnings.warn(


tensor([[0.5791, 0.5322]])
tensor(-2.9557)
-0.4344234176914185
tensor([[0.5300, 0.5425]])
tensor(-2.9772)
-0.1474072235969596
tensor([[1., 0.]])
tensor(-3.2050)
-0.1474072235969596
tensor([[0.5057, 0.4963]])
tensor(-2.7791)
-0.1474072235969596
tensor([[0.5147, 0.2444]])
tensor(-3.4794)
-0.1474072235969596
tensor([[0.5673, 0.7510]])
tensor(-3.7009)
-0.1474072235969596
tensor([[0.4294, 0.0000]])
tensor(-4.5426)
-0.1474072235969596
tensor([[0.4035, 0.3531]])
tensor(-2.9419)
-0.1474072235969596
tensor([[0.6213, 0.2835]])
tensor(-2.7978)
-0.1474072235969596
tensor([[0.5911, 0.9560]])
tensor(-2.9128)
-0.1474072235969596
tensor([[0., 0.]])
tensor(-3.0761)
-0.1474072235969596
tensor([[0.0365, 0.5428]])
tensor(-3.0757)
-0.1474072235969596
tensor([[0.5060, 0.8412]])
tensor(-3.0490)
-0.1474072235969596
tensor([[0.5535, 0.4912]])
tensor(-2.9397)
-0.1474072235969596
tensor([[0.9294, 0.5656]])
tensor(-3.8308)
-0.1474072235969596
tensor([[0.6459, 0.8525]])
tensor(-3.4208)
-0.1474072235969596
tensor([

/home/hanyang/anaconda3/envs/known_boundary/lib/python3.9/site-packages/botorch/optim/optimize.py:369: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)


tensor([[0.5183, 0.6706]])
tensor(-5.1334)
-0.1474072235969596
tensor([[0.0000, 0.8707]])
tensor(-9.1095)
-0.1474072235969596
tensor([[1.0000, 0.3458]])
tensor(-10.6085)
-0.1474072235969596
tensor([[0.7207, 0.4546]])
tensor(-2.9920)
-0.1474072235969596
tensor([[0.7204, 0.0000]])
tensor(-5.0406)
-0.1474072235969596
tensor([[0.5527, 0.4086]])
tensor(-7.8168)
-0.1474072235969596
tensor([[0.1956, 0.4339]])
tensor(-10.3906)
-0.1474072235969596
tensor([[0.7858, 0.1830]])
tensor(-9.3506)
-0.1474072235969596
tensor([[0.5387, 0.5625]])
tensor(-4.5606)
-0.05577618912907121
tensor([[1., 1.]])
tensor(-28.0602)
-0.05577618912907121
tensor([[0.4915, 0.9625]])
tensor(-14.3531)
-0.05577618912907121
tensor([[0.5325, 0.5531]])
tensor(-4.3839)
-0.05577618912907121
tensor([[0.7568, 0.5382]])
tensor(-26.5450)
-0.05577618912907121


In [53]:
print('EI: ',EI_record)

EI:  [array([-1.1237396 , -1.1237396 , -0.80903737, -0.80903737, -0.80903737,
       -0.80903737, -0.80903737, -0.80903737, -0.80903737, -0.80903737,
       -0.80903737, -0.80903737, -0.27803632, -0.27803632, -0.27803632,
       -0.27803632, -0.27803632, -0.27803632, -0.27803632, -0.27803632,
       -0.15624171, -0.15624171, -0.15624171, -0.15624171, -0.15624171,
       -0.15624171, -0.15624171, -0.15624171, -0.15624171, -0.15624171,
       -0.15624171])]


In [55]:
print('pdf gradient: ',EI_record)

pdf gradient:  [array([-1.1237396 , -0.79409405, -0.4033131 , -0.4033131 , -0.38387134,
       -0.38387134, -0.38387134, -0.38387134, -0.38387134, -0.38387134,
       -0.38387134, -0.38387134, -0.38387134, -0.38387134, -0.38387134,
       -0.38387134, -0.38387134, -0.38387134, -0.38387134, -0.38387134,
       -0.38387134, -0.38387134, -0.38387134, -0.38387134, -0.38387134,
       -0.38387134, -0.38387134, -0.38387134, -0.38387134, -0.38387134,
       -0.38387134])]


In [57]:
print('pdf gradient fansaty: ',EI_record)

pdf gradient fansaty:  [array([-1.1237396 , -0.43442342, -0.14740722, -0.14740722, -0.14740722,
       -0.14740722, -0.14740722, -0.14740722, -0.14740722, -0.14740722,
       -0.14740722, -0.14740722, -0.14740722, -0.14740722, -0.14740722,
       -0.14740722, -0.14740722, -0.14740722, -0.14740722, -0.14740722,
       -0.14740722, -0.14740722, -0.14740722, -0.14740722, -0.14740722,
       -0.14740722, -0.05577619, -0.05577619, -0.05577619, -0.05577619,
       -0.05577619])]
